In [1]:
import math
import numpy as np
import pandas as pd
import ortools
import itertools

from typing import Generator, List, Tuple

In [2]:
# look at all possible cut assignments that result in the needed quantities
#   minimize over all of the cut assignments
from ortools.sat.python import cp_model


In [3]:
fn = "./#5 Consolidated.csv"

In [4]:
df = pd.read_csv(fn, header=None)
df.columns = ["length", "quantity"]

base_length = 240
waste_tolerance = 0

exact_cuts = df[df["length"] == base_length].copy()

# df = df[df["length"] < base_length]
needed_lengths = df["length"].values
needed_quantities = df["quantity"].values

# We expect to be allowed to make cuts
# 
# allowed_cuts = [
#   (3,), (3, 3,), (3, 3, 3,), (3, 5,), (5,), (5, 3,), (5, 5,), (8,),
# ]

In [5]:
len(df)

19

In [6]:
df["quantity"].sum()

976

In [7]:
df.sort_values("length", ascending=False)

,length,quantity
7,240,232
13,184,6
11,168,16
12,162,8
14,148,6
10,132,88
6,127,45
5,120,48
8,116,8
17,108,16


In [8]:
def get_possible_cuts(needed_lengths: List[int], base_length: int, tolerance_per_cut: float = 0) -> Generator:
    
    # Establish a bound on the number of combinations that we need
    max_combinations = math.floor(base_length / min(needed_lengths))

    for i in range(1, max_combinations + 1):
        for comb in itertools.combinations_with_replacement(needed_lengths, i):
            waste = len(comb) * tolerance_per_cut
            if len(comb) == 1 and comb[0] == base_length:
                yield comb
                continue
                
            if sum(comb) <= base_length - waste:
                yield comb

In [9]:
possible_cuts = list(
    get_possible_cuts(needed_lengths, base_length, tolerance_per_cut=0)
)

In [10]:
len(possible_cuts)

605

In [11]:
cut_produced_quantities = {}
for i, cut in enumerate(possible_cuts):
    for j, length in enumerate(needed_lengths):
        cut_produced_quantities[i, j] = cut.count(length)
        
cut_wastes = {}
for i, cut in enumerate(possible_cuts):
    waste = base_length - sum(cut)    
    cut_wastes[i] = waste + .0001

In [12]:
model = cp_model.CpModel()
naive_max_cuts = max(needed_quantities)

cut_vars = {}
for i, cut in enumerate(possible_cuts):
    cut_vars[i] = model.NewIntVar(0, naive_max_cuts, f"Cut according to rule {cut}")

for (j, length), needed in zip(enumerate(needed_lengths), needed_quantities): 
    cut_sums = []
    for i, cut in enumerate(possible_cuts):
        cut_sums.append(cut_vars[i] * cut_produced_quantities[i, j])
        
    model.Add(sum(cut_sums) == needed)
    
# Let's minimize the cut wastest
total_cut_waste = sum(
    cut_vars[i] * cut_wastes[i] for i, _ in enumerate(possible_cuts)
)
model.Minimize(total_cut_waste)

In [13]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [26]:
solver.StatusName()

'OPTIMAL'

In [27]:
status

4

In [14]:
print(solver.ResponseStats())

CpSolverResponse summary:
status: OPTIMAL
objective: 1504.0469
best_bound: 1504.0469
booleans: 107
conflicts: 121
branches: 223
propagations: 925
integer_propagations: 21401
restarts: 15
lp_iterations: 0
walltime: 0.101831
usertime: 0.101831
deterministic_time: 0.0105985
gap_integral: 0.0686026



In [15]:
solution_df = []
for i, var in cut_vars.items():
    solution_df.append(
        {
            "cut": possible_cuts[i],
            "cut_idx": i,
            "number": solver.Value(var),
        }
    )
solution_df = pd.DataFrame(solution_df)
solution_df["waste"] = solution_df["cut_idx"].apply(cut_wastes.get)
solution_df["total_waste"] = solution_df["waste"] * solution_df["number"]

confirmed_counts = {}
# confirm that we didn't fuck it
for i, var in cut_vars.items():
    for j, length in enumerate(needed_lengths):
        confirmed_counts[length] = confirmed_counts.get(length, 0) + cut_produced_quantities[i, j] * solver.Value(var)
        
confirmed_counts_df = pd.DataFrame(
    [{"length": length, "quantity": quantity} for length, quantity in confirmed_counts.items()]
)

In [16]:
confirmatory_df = df.merge(
    confirmed_counts_df,
    how="left",
    on="length",
    suffixes=["", "_produced_by_algorithm"],
)

assert (confirmatory_df["quantity"] == confirmatory_df["quantity_produced_by_algorithm"]).all()

In [17]:
total_waste = solution_df["total_waste"].sum()
total_length_needed = np.sum(df["length"] * df["quantity"])

In [18]:
total_waste

1504.0469

In [19]:
solution_df["number"].sum()

469

In [20]:
efficiency = 1 - total_waste / total_length_needed

In [21]:
print(f"We are {efficiency *100:.2f}% efficient")

We are 98.65% efficient


In [22]:
# Let's model this using a simple heuristic

In [23]:
solved_fn = fn.replace(".csv", "-solved.csv")

In [24]:
solution_df.to_csv(solved_fn)
print(f"wrote to {solved_fn}")

wrote to ./#5 Consolidated-solved.csv
